# Binary Model

## Extracting training data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import os
import csv
from docx import Document
import json
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,accuracy_score, precision_recall_curve,f1_score, confusion_matrix, RocCurveDisplay
from sklearn.metrics import classification_report

In [4]:
merged_df=pd.read_csv('Merge_data.csv')

In [5]:
merged_df

,type,status,departure.iataCode,departure.icaoCode,departure.terminal,departure.scheduledTime,departure.estimatedTime,departure.actualTime,departure.estimatedRunway,departure.actualRunway,...,Wind Speed Min,Pressure Max,Pressure Avg,Pressure Min,Total,delay,delay_in_minutes,day_of_week,hour_of_day,month_of_year
0,departure,active,lhe,opla,m,2023-07-20 20:50:00,2023-07-20t20:00:00.000,2023-07-20 20:15:00,2023-07-20t20:15:00.000,2023-07-20t20:15:00.000,...,0,28.9,28.8,28.7,0.0,-1 days +23:25:00,0.000000,Thursday,20,July
1,departure,active,lhe,opla,NaN,2023-07-18 15:05:00,2023-07-18t15:05:00.000,NaN,NaN,NaN,...,0,28.8,28.8,28.7,0.0,NaN,11.611388,Tuesday,15,July
2,departure,active,lhe,opla,NaN,2023-07-23 09:50:00,NaN,NaN,NaN,NaN,...,0,29.0,28.9,28.8,0.0,NaN,11.611388,Sunday,9,July
3,departure,active,lhe,opla,m,2023-07-26 23:30:00,2023-07-26t23:30:00.000,2023-07-26 23:51:00,2023-07-26t23:51:00.000,2023-07-26t23:51:00.000,...,0,28.9,28.9,28.8,0.0,0 days 00:21:00,21.000000,Wednesday,23,July
4,departure,active,lhe,opla,m,2023-07-20 11:35:00,2023-07-20t17:15:00.000,NaN,NaN,NaN,...,0,28.9,28.8,28.7,0.0,NaN,11.611388,Thursday,11,July
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12109,departure,active,khi,opkc,m,2023-08-02 18:40:00,2023-08-02t18:44:00.000,2023-08-02 18:54:00,2023-08-02t18:54:00.000,2023-08-02t18:54:00.000,...,0,28.8,28.8,28.7,0.0,0 days 00:14:00,14.000000,Wednesday,18,August
12110,departure,active,khi,opkc,m,2023-08-10 04:00:00,2023-08-10t04:00:00.000,2023-08-10 03:58:00,2023-08-10t03:58:00.000,2023-08-10t03:58:00.000,...,0,28.9,28.8,28.8,0.0,-1 days +23:58:00,0.000000,Thursday,4,August
12111,departure,active,khi,opkc,m,2023-08-01 16:35:00,2023-08-01t16:35:00.000,2023-08-01 17:10:00,2023-08-01t17:10:00.000,2023-08-01t17:10:00.000,...,5,28.8,28.8,28.7,0.0,0 days 00:35:00,35.000000,Tuesday,16,August
12112,departure,cancelled,khi,opkc,m,2023-08-12 07:00:00,NaN,NaN,NaN,NaN,...,0,28.9,28.8,28.7,0.0,NaN,11.611388,Saturday,7,August


## Hot encoding

In [6]:

df_encoded = pd.get_dummies(merged_df, columns=['status','type','month_of_year','day_of_week','hour_of_day','airline.name',
       'airline.iataCode', 'airline.icaoCode', 'flight.number',
       'flight.iataNumber', 'flight.icaoNumber',])
df_encoded

,departure.iataCode,departure.icaoCode,departure.terminal,departure.scheduledTime,departure.estimatedTime,departure.actualTime,departure.estimatedRunway,departure.actualRunway,arrival.iataCode,arrival.icaoCode,...,flight.icaoNumber_uae603,flight.icaoNumber_uae605,flight.icaoNumber_uae607,flight.icaoNumber_uae623,flight.icaoNumber_uae625,flight.icaoNumber_uae8619,flight.icaoNumber_uae8621,flight.icaoNumber_voz6098,flight.icaoNumber_voz6173,flight.icaoNumber_vsv850
0,lhe,opla,m,2023-07-20 20:50:00,2023-07-20t20:00:00.000,2023-07-20 20:15:00,2023-07-20t20:15:00.000,2023-07-20t20:15:00.000,ruh,oerk,...,0,0,0,0,0,0,0,0,0,0
1,lhe,opla,NaN,2023-07-18 15:05:00,2023-07-18t15:05:00.000,NaN,NaN,NaN,khi,opkc,...,0,0,0,0,0,0,0,0,0,0
2,lhe,opla,NaN,2023-07-23 09:50:00,NaN,NaN,NaN,NaN,khi,opkc,...,0,0,0,0,0,0,0,0,0,0
3,lhe,opla,m,2023-07-26 23:30:00,2023-07-26t23:30:00.000,2023-07-26 23:51:00,2023-07-26t23:51:00.000,2023-07-26t23:51:00.000,kwi,okkk,...,0,0,0,0,0,0,0,0,0,0
4,lhe,opla,m,2023-07-20 11:35:00,2023-07-20t17:15:00.000,NaN,NaN,NaN,dxb,omdb,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12109,khi,opkc,m,2023-08-02 18:40:00,2023-08-02t18:44:00.000,2023-08-02 18:54:00,2023-08-02t18:54:00.000,2023-08-02t18:54:00.000,auh,omaa,...,0,0,0,0,0,0,0,0,0,0
12110,khi,opkc,m,2023-08-10 04:00:00,2023-08-10t04:00:00.000,2023-08-10 03:58:00,2023-08-10t03:58:00.000,2023-08-10t03:58:00.000,doh,othh,...,0,0,0,0,0,0,0,0,0,0
12111,khi,opkc,m,2023-08-01 16:35:00,2023-08-01t16:35:00.000,2023-08-01 17:10:00,2023-08-01t17:10:00.000,2023-08-01t17:10:00.000,dxb,omdb,...,0,0,0,0,0,0,0,0,0,0
12112,khi,opkc,m,2023-08-12 07:00:00,NaN,NaN,NaN,NaN,isb,opis,...,0,0,0,0,0,0,0,0,0,0


## Normalizing data

In [7]:
from sklearn.preprocessing import MinMaxScaler
columns_to_normalize=['Temp Max','Temp Avg', 'Temp Min', 'Dew Point Max', 'Dew Point Avg',
       'Dew Point Min', 'Humidity Max', 'Humidity Avg', 'Humidity Min',
       'Wind Speed Max', 'Wind Speed Avg', 'Wind Speed Min', 'Pressure Max',
       'Pressure Avg', 'Pressure Min']
scaler = MinMaxScaler()


df_encoded[columns_to_normalize] = scaler.fit_transform(df_encoded[columns_to_normalize])

## Mapping and splitting data

In [8]:

for index, i in enumerate(merged_df['delay_in_minutes']):

    if i == 0.0:
        merged_df.loc[index, 'delay_status'] = 1
    else:
        merged_df.loc[index, 'delay_status'] = 0


from sklearn.model_selection import train_test_split

X = df_encoded[['type_departure', 'month_of_year_April', 'month_of_year_August',
       'month_of_year_December', 'month_of_year_February',
       'month_of_year_January', 'month_of_year_July', 'month_of_year_June',
       'month_of_year_March', 'month_of_year_May', 'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday',
       'hour_of_day_0', 'hour_of_day_1', 'hour_of_day_2', 'hour_of_day_3',
       'hour_of_day_4', 'hour_of_day_5', 'hour_of_day_6', 'hour_of_day_7',
       'hour_of_day_8', 'hour_of_day_9', 'hour_of_day_10', 'hour_of_day_11',
       'hour_of_day_12', 'hour_of_day_13', 'hour_of_day_14', 'hour_of_day_15',
       'hour_of_day_16', 'hour_of_day_17', 'hour_of_day_18', 'hour_of_day_19',
       'hour_of_day_20', 'hour_of_day_21', 'hour_of_day_22', 'hour_of_day_23','status_unknown','status_cancelled','status_active','Temp Max','Temp Avg', 'Temp Min', 'Dew Point Max', 'Dew Point Avg',
       'Dew Point Min', 'Humidity Max', 'Humidity Avg', 'Humidity Min',
       'Wind Speed Max', 'Wind Speed Avg', 'Wind Speed Min', 'Pressure Max',
       'Pressure Avg', 'Pressure Min']]
y = merged_df['delay_status']
y=y.astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Training model by Random Forest

In [11]:
binary_classifier = RandomForestClassifier(random_state=42)
binary_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

## Predicting 

In [14]:
y_valid_pred = binary_classifier.predict(X_test)
print("Validation Accuracy:", accuracy_score(y_test, y_valid_pred))
print("Classification Report on Validation Data:\n", classification_report(y_test, y_valid_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_valid_pred))

Validation Accuracy: 0.9087907552620718
Classification Report on Validation Data:
               precision    recall  f1-score   support

           0       0.94      0.96      0.95      2148
           1       0.63      0.48      0.54       275

    accuracy                           0.91      2423
   macro avg       0.78      0.72      0.75      2423
weighted avg       0.90      0.91      0.90      2423

Confusion Matrix:
 [[2070   78]
 [ 143  132]]


## Reading test data

In [15]:
test_df=pd.read_csv('Merge_data_test.csv')

In [16]:
test_df

,type,status,departure.iataCode,departure.icaoCode,departure.scheduledTime,departure.estimatedRunway,departure.actualRunway,arrival.iataCode,arrival.icaoCode,arrival.scheduledTime,...,Wind Speed Min,Pressure Max,Pressure Avg,Pressure Min,Total,delay,delay_in_minutes,day_of_week,hour_of_day,month_of_year
0,departure,active,lhe,opla,2023-07-17 20:35:00,2023-07-17t20:46:00.000,2023-07-17 20:46:00,khi,opkc,2023-07-17t22:20:00.000,...,0,28.8,28.8,28.7,0.0,0 days 00:11:00,11.000000,Monday,20,July
1,departure,active,lhe,opla,2023-07-27 08:00:00,NaN,NaN,ruh,oerk,2023-07-27t10:00:00.000,...,0,28.9,28.9,28.8,0.0,NaN,13.372743,Thursday,8,July
2,departure,active,lhe,opla,2023-07-27 08:00:00,NaN,NaN,dxb,omdb,2023-07-27t10:00:00.000,...,0,28.9,28.9,28.8,0.0,NaN,13.372743,Thursday,8,July
3,departure,unknown,lhe,opla,2023-07-28 16:45:00,NaN,NaN,jed,oejn,2023-07-28t20:30:00.000,...,0,28.8,28.8,28.7,0.0,NaN,13.372743,Friday,16,July
4,departure,active,lhe,opla,2023-07-19 04:15:00,2023-07-19t04:18:00.000,2023-07-19 04:18:00,auh,omaa,2023-07-19t06:35:00.000,...,0,28.8,28.8,28.7,0.0,0 days 00:03:00,3.000000,Wednesday,4,July
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14905,departure,active,lhe,opla,2023-11-23 23:15:00,2023-11-23t23:27:00.000,2023-11-23 23:27:00,shj,omsj,2023-11-24t02:05:00.000,...,0,29.0,28.9,28.8,0.0,0 days 00:12:00,12.000000,Thursday,23,November
14906,departure,active,lhe,opla,2023-11-25 18:45:00,2023-11-25t19:05:00.000,2023-11-25 19:05:00,jed,oejn,2023-11-25t22:05:00.000,...,0,28.9,28.9,28.8,0.0,0 days 00:20:00,20.000000,Saturday,18,November
14907,departure,active,lhe,opla,2023-11-25 02:55:00,2023-11-25t03:07:00.000,2023-11-25 03:07:00,doh,othh,2023-11-25t05:05:00.000,...,0,28.9,28.9,28.8,0.0,0 days 00:12:00,12.000000,Saturday,2,November
14908,departure,active,lhe,opla,2023-11-27 08:35:00,NaN,NaN,doh,othh,2023-11-27t10:45:00.000,...,0,28.9,28.9,28.8,0.0,NaN,13.372743,Monday,8,November


## Hot Encoding test

In [17]:

df_encoded_test = pd.get_dummies(test_df, columns=['status','type','month_of_year','day_of_week','hour_of_day','airline.name',
       'airline.iataCode', 'airline.icaoCode', 'flight.number',
       'flight.iataNumber', 'flight.icaoNumber'])
df_encoded_test.columns

Index(['departure.iataCode', 'departure.icaoCode', 'departure.scheduledTime',
       'departure.estimatedRunway', 'departure.actualRunway',
       'arrival.iataCode', 'arrival.icaoCode', 'arrival.scheduledTime',
       'arrival.estimatedTime', 'departure.terminal',
       ...
       'flight.icaoNumber_uae613', 'flight.icaoNumber_uae615',
       'flight.icaoNumber_uae623', 'flight.icaoNumber_uae625',
       'flight.icaoNumber_uae8619', 'flight.icaoNumber_uae8621',
       'flight.icaoNumber_voz6098', 'flight.icaoNumber_voz6173',
       'flight.icaoNumber_voz6174', 'flight.icaoNumber_vsv850'],
      dtype='object', length=1972)

## Predicting on basis of test

In [18]:
test_X = df_encoded_test[['type_departure', 'month_of_year_April', 'month_of_year_August',
       'month_of_year_December', 'month_of_year_February',
       'month_of_year_January', 'month_of_year_July', 'month_of_year_June',
       'month_of_year_March', 'month_of_year_May', 'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday',
       'hour_of_day_0', 'hour_of_day_1', 'hour_of_day_2', 'hour_of_day_3',
       'hour_of_day_4', 'hour_of_day_5', 'hour_of_day_6', 'hour_of_day_7',
       'hour_of_day_8', 'hour_of_day_9', 'hour_of_day_10', 'hour_of_day_11',
       'hour_of_day_12', 'hour_of_day_13', 'hour_of_day_14', 'hour_of_day_15',
       'hour_of_day_16', 'hour_of_day_17', 'hour_of_day_18', 'hour_of_day_19',
       'hour_of_day_20', 'hour_of_day_21', 'hour_of_day_22', 'hour_of_day_23','status_unknown','status_cancelled','status_active','Temp Max','Temp Avg', 'Temp Min', 'Dew Point Max', 'Dew Point Avg',
       'Dew Point Min', 'Humidity Max', 'Humidity Avg', 'Humidity Min',
       'Wind Speed Max', 'Wind Speed Avg', 'Wind Speed Min', 'Pressure Max',
       'Pressure Avg', 'Pressure Min']]


test_pred = binary_classifier.predict(test_X)


In [19]:
test_pred_labels = ["on-time" if pred == 0 else "delayed" for pred in test_pred]

In [20]:
submission_df = pd.DataFrame({
    'ID': range(1, len(test_pred) + 1),  
    'Delay': test_pred_labels  
})
sahal=submission_df[:12914]

In [21]:
sahal.to_csv("submission_binary.csv", index=False)
print("Submission file created: submission_binary.csv")

Submission file created: submission_binary.csv


In [22]:
sahal['Delay'].value_counts()

on-time    12833
delayed       81
Name: Delay, dtype: int64